## Setup

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image

import imageio
import torch as th
import tqdm

from imitation_modules import NonImageCnnRewardNet
from stealing_gridworld import StealingGridworld
from value_iteration import get_optimal_policy

/Users/davisfoote/opt/anaconda3/envs/assisting_bounded_humans/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [75]:
visibilities = ["full", "partial"]
model_paths = [
    "saved_reward_models/full-vis_scalar_reward_model_5_32,32_3_20230425_011443/latest_checkpoint.pt",
    "saved_reward_models/partial-vis_scalar_reward_model_5_32,32_3_20230424_210742/latest_checkpoint.pt",
]

GRID_SIZE = 5
HORIZON = 30

HID_CHANNELS = (32, 32)
KERNEL_SIZE = 3


visibility_mask = np.array([
    [0, 0, 0, 0, 0],
    [0, 1, 1, 1, 0],
    [0, 1, 1, 1, 0],
    [0, 1, 1, 1, 0],
    [0, 0, 0, 0, 0],
])


env = StealingGridworld(
    grid_size=GRID_SIZE,
    max_steps=HORIZON,
    reward_for_depositing=100,
    reward_for_picking_up=1,
    reward_for_stealing=-200,
)


def load_model_params(model_path):
    reward_net = NonImageCnnRewardNet(
        env.observation_space,
        env.action_space,
        hid_channels=HID_CHANNELS,
        kernel_size=KERNEL_SIZE,
    )
    model_state_dict = th.load(model_path, map_location=th.device('cpu'))
    reward_net.load_state_dict(model_state_dict)
    return reward_net


reward_nets = [load_model_params(model_path) for model_path in model_paths]
policies = [get_optimal_policy(env, alt_reward_fn=reward_net) for reward_net in reward_nets]

Value iteration: 100%|██████████| 30/30 [00:01<00:00, 23.26it/s]


In [36]:
# Load and size PNGs per entity

img_dir = "presentation/images/stealy_dan"

GRID_CELL_IMAGE_SHAPE = (400, 400, 4)

def pad_image_to_shape(image, bias="right", shape=GRID_CELL_IMAGE_SHAPE):
    missing_rows = shape[0] - image.shape[0]
    missing_cols = shape[1] - image.shape[1]
    top_pad = missing_rows // 2
    bottom_pad = missing_rows - top_pad
    if bias == "left":
        left_pad = missing_cols // 12
        right_pad = missing_cols - left_pad
    elif bias == "right":
        right_pad = missing_cols // 12
        left_pad = missing_cols - right_pad
    return np.pad(image, ((top_pad, bottom_pad), (left_pad, right_pad), (0, 0)), mode="constant")


# Load PNGs for each entity
agent_pngs = [pad_image_to_shape(plt.imread(f"{img_dir}/agent_{i}.png"), bias="left") for i in range(4)]
free_pellet_png = pad_image_to_shape(plt.imread(f"{img_dir}/free_pellet.png"))
owned_pellet_png = pad_image_to_shape(plt.imread(f"{img_dir}/owned_pellet.png"))
home_png = pad_image_to_shape(plt.imread(f"{img_dir}/home.png"))

In [34]:
def get_image_from_state(state, visibility_mask=None):
    grid_size = state.shape[1]

    opacity_mask = np.ones((grid_size, grid_size))
    opacity_mask[np.where(visibility_mask == 0)] = 0.5

    full_grid = np.zeros((grid_size * GRID_CELL_IMAGE_SHAPE[0], grid_size * GRID_CELL_IMAGE_SHAPE[1], 4))
    full_grid[:, :] = [0.7, 1.0, 0.7, 1.0]

    for i in range(grid_size):
        for j in range(grid_size):
            cell = state[:, i, j]
            cell_png = np.zeros(GRID_CELL_IMAGE_SHAPE)
            if cell[0] == 1:
                cell_png += agent_pngs[cell[-1]]
            if cell[1] == 1:
                cell_png += free_pellet_png
            if cell[2] == 1:
                cell_png += owned_pellet_png
            if cell[3] == 1:
                cell_png += home_png
            # Only overwrite pixels that are not transparent
            full_grid[
                i * GRID_CELL_IMAGE_SHAPE[0] : (i + 1) * GRID_CELL_IMAGE_SHAPE[0],
                j * GRID_CELL_IMAGE_SHAPE[1] : (j + 1) * GRID_CELL_IMAGE_SHAPE[1],
                :,
            ] = np.where(cell_png[..., -1:] > 0, cell_png, full_grid[
                i * GRID_CELL_IMAGE_SHAPE[0] : (i + 1) * GRID_CELL_IMAGE_SHAPE[0],
                j * GRID_CELL_IMAGE_SHAPE[1] : (j + 1) * GRID_CELL_IMAGE_SHAPE[1],
                :,
            ])
            if visibility_mask is not None:
                full_grid[
                    i * GRID_CELL_IMAGE_SHAPE[0] : (i + 1) * GRID_CELL_IMAGE_SHAPE[0],
                    j * GRID_CELL_IMAGE_SHAPE[1] : (j + 1) * GRID_CELL_IMAGE_SHAPE[1],
                    :,
                ] *= opacity_mask[i, j]

    # Draw the grid lines
    thickness = 3
    for i in range(grid_size):
        full_grid[i * GRID_CELL_IMAGE_SHAPE[0], :, :] = 0
        full_grid[i * GRID_CELL_IMAGE_SHAPE[0] - thickness : i * GRID_CELL_IMAGE_SHAPE[0] + thickness, :, :] = 0
        full_grid[:, i * GRID_CELL_IMAGE_SHAPE[1], :] = 0
        full_grid[:, i * GRID_CELL_IMAGE_SHAPE[1] - thickness : i * GRID_CELL_IMAGE_SHAPE[1] + thickness, :] = 0
    # Outer border
    full_grid[:thickness*2, :, :] = 0
    full_grid[-thickness*2:, :, :] = 0
    full_grid[:, :thickness*2, :] = 0
    full_grid[:, -thickness*2:, :] = 0

    return full_grid

In [83]:
def save_image_from_states(states, output_file, visibility_mask=None, frame_rate=4):
    pil_images = []
    for state in states:
        image = get_image_from_state(state, visibility_mask)
        image = (image[:, :, :3] * 255).astype(np.uint8)
        pil_images.append(Image.fromarray(image))
    if len(pil_images) == 1:
        pil_images[0].save(output_file)
    else:
        pil_images[0].save(
            output_file,
            save_all=True,
            append_images=pil_images[1:],
            duration=int(1000 / frame_rate),
            loop=0,
        )

## Stills

In [47]:
stills_dir = "presentation/images/env_stills"

In [93]:
env.reset()
env.pellet_locations = {
    "free": np.array([[2, 3], [3, 2]]),
    "owned": np.array([[4, 3]]),
}
env.agent_position = np.array([1, 2])
save_image_from_states([env._get_observation()], f"{stills_dir}/explain_env.png")

In [54]:
env.reset()
env.pellet_locations = {
    "free": np.array([]),
    "owned": np.array([[4, 3]]),
}
env.num_carried_pellets = 1
save_image_from_states([env._get_observation()], f"{stills_dir}/good_action.png")

In [58]:
env.reset()
env.pellet_locations = {
    "free": np.array([]),
    "owned": np.array([[1, 3]]),
}
env.agent_position = np.array([1, 3])
save_image_from_states([env._get_observation()], f"{stills_dir}/bad_action.png")

In [60]:
env.reset()
env.pellet_locations = {
    "free": np.array([]),
    "owned": np.array([[4, 3]]),
}
env.agent_position = np.array([4, 3])
save_image_from_states([env._get_observation()], f"{stills_dir}/bad_action_hidden.png")

In [95]:
env.reset()
env.pellet_locations = {
    "free": np.array([]),
    "owned": np.array([]),
}
save_image_from_states([env._get_observation()], f"{stills_dir}/masked.png", visibility_mask=visibility_mask)

## Videos

In [61]:
videos_dir = "presentation/videos"

In [80]:
from evaluate_reward_model import full_visibility_evaluator_factory, partial_visibility_evaluator_factory

full_vis_evaluator = full_visibility_evaluator_factory()
partial_vis_evaluator = partial_visibility_evaluator_factory(visibility_mask)

# These evaluate that the policies behave as expected (not necesarily optimally).
# Everything should be 0 or close to 0.
print(full_vis_evaluator.evaluate(policies[0], env, num_trajs=1000))
print(partial_vis_evaluator.evaluate(policies[1], env, num_trajs=1000))

Rollouts for evaluation: 100%|██████████| 1000/1000 [00:05<00:00, 184.94it/s]


(0.0, {StoleVisiblePellet: 0.0, FailedToPickUpFreePellet: 0.0, FailedToDepositPellet: 0.0})


Rollouts for evaluation: 100%|██████████| 1000/1000 [00:06<00:00, 148.52it/s]

(0.086, {StoleVisiblePellet: 0.0, DidntStealInvisiblePellet: 0.053, FailedToPickUpFreePellet: 0.072, FailedToDepositPellet: 0.0})


In [81]:
output_file = f"{videos_dir}/well_behaved.gif"

num_rollouts = 5

trajs = [env.rollout_with_policy(policies[0], render=False) for _ in range(num_rollouts)]

states = []
for traj in trajs:
    states.extend(traj.obs)

save_image_from_states(states, output_file)

In [88]:
output_file = f"{videos_dir}/stealing_off_cam.gif"
output_file_masked = f"{videos_dir}/stealing_off_cam_masked.gif"

num_rollouts = 5

trajs = [env.rollout_with_policy(policies[1], render=False) for _ in range(num_rollouts)]

states = []
for traj in trajs:
    states.extend(traj.obs)

save_image_from_states(states, output_file_masked, visibility_mask=visibility_mask)
save_image_from_states(states, output_file)